In [1]:
import os
import tensorflow as tf
from PIL import Image
import PIL
import numpy as np

In [2]:
folder_path = os.getcwd()
ws_path = os.path.dirname(folder_path)
ws_path

'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification'

In [3]:
list_dir = os.listdir(ws_path)
frame_path = ws_path + '/rawFrame'
prepro_data_path = ws_path + '/preproDataset'
nor_fr_path = frame_path + '/normal'
vsd_fr_path = frame_path + '/vsd'
nor_prepro_path = prepro_data_path + '/normal'
vsd_prepro_path = prepro_data_path + '/vsd'

In [9]:
# Create output directory
os.makedirs(prepro_data_path, exist_ok=True)
os.makedirs(nor_prepro_path, exist_ok=True)
os.makedirs(vsd_prepro_path, exist_ok=True)

In [10]:
nor_img = os.listdir(nor_fr_path)
nor_fr_list_path = list()

for img in nor_img:
    path = nor_fr_path + f'/{img}'
    nor_fr_list_path.append(path)

nor_fr_list_path

['e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0001.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0002.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0003.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0004.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0005.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0006.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0007.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/normal/m0_fr0008.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Di

In [11]:
vsd_img = os.listdir(vsd_fr_path)
vsd_fr_list_path = list()

for img in vsd_img:
    path = vsd_fr_path + f'/{img}'
    vsd_fr_list_path.append(path)

vsd_fr_list_path

['e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0001.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0002.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0003.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0004.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0005.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0006.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0007.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unity\\Tugas\\Heart-Leaks-Disease-Classification/rawFrame/vsd/25212_m3_fr0008.png',
 'e:\\Kuliah\\MBKM\\CV Advance With Unit

In [12]:
def preprocess(input_path, output_path, img_size=(224, 224)):
    for path in input_path:
        img = tf.keras.preprocessing.image.load_img(path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)

        img_array = img_array / 255.0
        img_uint8 = tf.image.convert_image_dtype(img_array, dtype=tf.uint8)

        filename = os.path.basename(path)
        filename = os.path.splitext(filename)[0] + '.png'

        output_file = os.path.join(output_path, filename)

        tf.keras.utils.save_img(output_file, img_uint8)


In [13]:
preprocess(nor_fr_list_path, nor_prepro_path)
preprocess(vsd_fr_list_path, vsd_prepro_path)